In [12]:
import requests
from bs4 import BeautifulSoup
import MySQLdb
import sys

In [14]:
# 웹 크롤링 한 결과를 저장할 리스트
lotto_list = []

# 로또 웹 사이트의 첫 주소
main_url = "https://www.nlotto.co.kr/lotto645Confirm.do?method=byWin"

# 각 회차별 당첨정보를 알 수 있는 주소
basic_url = "https://www.nlotto.co.kr/lotto645Confirm.do?method=byWin&drwNo="


def getLast():
    resp = requests.get(main_url)
    soup = BeautifulSoup(resp.text, "lxml")
    line = str(soup.find("meta", {"id": "desc", "name": "description"})['content'])

    begin = line.find(" ")
    end = line.find("회")

    if begin == -1 or end == -1:
        print("not found last lotto number")
        exit()

    return int(line[begin + 1: end])


def checkLast():
    db = MySQLdb.connect(host="localhost", user="lotto", passwd="lotto", db="lotto")
    cursor = db.cursor()

    sql = "SELECT MAX(count) FROM data"
    try:
        cursor.execute(sql)
        result = cursor.fetchone()
    except:
        print(sys.exc_info()[0])

    cursor.close()
    db.close()

    return result[0]


def crawler(fromPos, toPos):
    for i in range(fromPos + 1, toPos + 1):
        crawler_url = basic_url + str(i)
        print("crawler: " + crawler_url)

        resp = requests.get(crawler_url)
        soup = BeautifulSoup(resp.text, "lxml")
        line = str(soup.find("meta", {"id": "desc", "name": "description"})['content'])

        begin = line.find("당첨번호")
        begin = line.find(" ", begin) + 1
        end = line.find(".", begin)
        numbers = line[begin:end]

        begin = line.find("총")
        begin = line.find(" ", begin) + 1
        end = line.find("명", begin)
        persons = line[begin:end]

        begin = line.find("당첨금액")
        begin = line.find(" ", begin) + 1
        end = line.find("원", begin)
        amount = line[begin:end]

        info = {}
        info["회차"] = i
        info["번호"] = numbers
        info["당첨자"] = persons
        info["금액"] = amount

        lotto_list.append(info)


def insert():
    db = MySQLdb.connect(host="localhost", user="lotto", passwd="lotto", db="lotto")
    cursor = db.cursor()

    for dic in lotto_list:
        count = dic["회차"]
        numbers = dic["번호"]
        persons = dic["당첨자"]
        amounts = dic["금액"]

        print("insert to database at " + str(count))

        numberlist = str(numbers).split(",")

        sql = "INSERT INTO `lotto`. `data`(`count`, `1`, `2`, `3`, `4`, `5`, `6`, `7`, `persion`, `amount`) " \
              "VALUES('%d', '%d', '%d', '%d', '%d', '%d', '%d', '%d', '%d', '%s')" \
              % (count,
                 int(numberlist[0]),
                 int(numberlist[1]),
                 int(numberlist[2]),
                 int(numberlist[3]),
                 int(numberlist[4]),
                 int(numberlist[5].split("+")[0]),
                 int(numberlist[5].split("+")[1]),
                 int(persons),
                 str(amounts))

        try:
            cursor.execute(sql)
            db.commit()
        except:
            print(sys.exc_info()[0])
            db.rollback()
            break

    cursor.close()
    db.close()


def analysis():
    db = MySQLdb.connect(host="localhost", user="lotto", passwd="lotto", db="lotto")
    cursor = db.cursor()

    for i in range(1, 8):

        sql = "select `"
        sql += str(i)
        sql += "` from data"

        try:
            cursor.execute(sql)
            results = cursor.fetchall()

            # 해당 숫자의 뽑힌 횟수를 하나씩 증가
            myarray = [0 for i in range(46)]
            for row in results:
                k = row[0]
                count = myarray[k]
                myarray[k] = count + 1
            print(i, myarray.index(max(myarray)))
        except:
            print(sys.exc_info()[0])

    cursor.close()
    db.close()

def distance():
    db = MySQLdb.connect(host="localhost", user="lotto", passwd="lotto", db="lotto")
    cursor = db.cursor()

    try:
        cursor.execute("select * from data order by `1` asc")
        results = cursor.fetchall()

        for row in results:
            print("1'st", row[1], "| ", end="")

            for i in range(1, 7):
                print(abs(row[i + 1] - row[i]), " ", end="")

            print(" ")

    except:
        print(sys.exc_info()[0])

    cursor.close()
    db.close()


def main():
    last = getLast()
    dblast = checkLast()

    if dblast < last:
        print("최신 회차는 " + str(last) + " 회 이며, 데이터베이스에는 " + str(dblast) + "회 까지 저장되어 있습니다.")
        print("업데이트를 시작합니다.")
        crawler(dblast, last)

    insert()
    #analysis()
    distance()

if __name__ == "__main__":
    main()


def graph():
    graph = pydotplus.Dot(graph_type='digraph')

    # in the last example, we did no explicitly create nodes, we just created the edges and
    # they automatically placed nodes on the graph. Unfortunately, this way we cannot specify
    # custom styles for the nodes (although you CAN set a default style for all objects on
    # the graph...), so let's create the nodes manually.

    # creating nodes is as simple as creating edges!
    node_a = pydotplus.Node("Node A", style="filled", fillcolor="red")
    # but... what are all those extra stuff after "Node A"?
    # well, these arguments define how the node is going to look on the graph,
    # you can find a full reference here:
    # http://www.graphviz.org/doc/info/attrs.html
    # which in turn is part of the full docs in
    # http://www.graphviz.org/Documentation.php

    # neat, huh? Let us create the rest of the nodes!
    node_b = pydotplus.Node("Node B", style="filled", fillcolor="green")
    node_c = pydotplus.Node("Node C", style="filled", fillcolor="#0000ff")
    node_d = pydotplus.Node("Node D", style="filled", fillcolor="#976856")

    # ok, now we add the nodes to the graph
    graph.add_node(node_a)
    graph.add_node(node_b)
    graph.add_node(node_c)
    graph.add_node(node_d)

    # and finally we create the edges
    # to keep it short, I'll be adding the edge automatically to the graph instead
    # of keeping a reference to it in a variable
    graph.add_edge(pydotplus.Edge(node_a, node_b))
    graph.add_edge(pydotplus.Edge(node_b, node_c))
    graph.add_edge(pydotplus.Edge(node_c, node_d))
    # but, let's make this last edge special, yes?
    graph.add_edge(pydotplus.Edge(node_d, node_a, label="and back we go again", labelfontcolor="#009933", fontsize="10.0",
                              color="blue"))

    # and we are done
    graph.write_png('example2_graph.png')

    graph = pydotplus.Dot(graph_type='graph')

    # the idea here is not to cover how to represent the hierarchical data
    # but rather how to graph it, so I'm not going to work on some fancy
    # recursive function to traverse a multidimensional array...
    # I'm going to hardcode stuff... sorry if that offends you

    # let's add the relationship between the king and vassals
    for i in range(3):
        # we can get right into action by "drawing" edges between the nodes in our graph
        # we do not need to CREATE nodes, but if you want to give them some custom style
        # then I would recomend you to do so... let's cover that later
        # the pydot.Edge() constructor receives two parameters, a source node and a destination
        # node, they are just strings like you can see
        edge = pydotplus.Edge("king", "lord%d" % i)
        # and we obviosuly need to add the edge to our graph
        graph.add_edge(edge)

    # now let us add some vassals
    vassal_num = 0
    for i in range(3):
        # we create new edges, now between our previous lords and the new vassals
        # let us create two vassals for each lord
        for j in range(2):
            edge = pydotplus.Edge("lord%d" % i, "vassal%d" % vassal_num)
            graph.add_edge(edge)
            vassal_num += 1

    # ok, we are set, let's save our graph into a file
    graph.write_png('example1_graph.png')

TypeError: 'NoneType' object is not subscriptable